In [1]:
import pandas as pd
from datetime import datetime, timedelta
import yfinance as yf  # https://pypi.org/project/yfinance/

In [76]:
NO_YEARS = 20
DAILY_INVEST = 10000 # €100
TICKERS = [
    "AIAG.L", # L&G Artificial Intelligence UCITS ETF (AIAG.L)
    "NBTK.DE",  # Invesco NASDAQ Biotech UCITS ETF (NBTK.DE)
    "XMLH.DE", # L&G Healthcare Breakthrough UCITS ETF (XMLH.DE)
    "ROBO", # Robo Global Robotics and Automation Index ETF (ROBO)
    "TIUP.DE", # Lyxor Core US TIPS (DR) UCITS ETF (TIUP.DE)
    "VWRL.AS", # Vanguard FTSE All-World UCITS ETF (VWRL.AS)
    "VFEM.AS", # Vanguard FTSE Emerging Markets UCITS ETF (VFEM.AS)
    "DXSA.DE", # Xtrackers Euro Stoxx Quality Dividend UCITS ETF (DXSA.DE)
    "SXR8.DE", # iShares VII PLC - iShares Core S&P 500 UCITS ETF (SXR8.DE)
    "IPRP.AS", # iShares European Property Yield UCITS ETF (IPRP.AS)
    "2B78.DE", # iShares Healthcare Innovation UCITS ETF (2B78.DE)
]

In [77]:
dfs = []
dates = set() # Keep track of distinct dates in all DataFrames.
# Create a list of DataFrames
for tick in TICKERS:
    df = yf.Ticker(tick).history(period=f"{NO_YEARS}y")
    df["Change"] = df["Close"].pct_change()*100
    df.drop(index=df.index[0], axis=0, inplace=True)  # no change on 1st day
    df.reset_index(inplace=True)  # make idx a col
    df = df[["Date", "Close", "Change"]]
    df["Tick"] = tick
    df["Date"] = df["Date"].dt.date
    dates = dates | set(df["Date"])
    dfs.append(df)

In [78]:
# Make one DataFrame with all tickers' data so calculations can be vectorized:
df = pd.DataFrame(index=sorted(dates))
for ticker_df in dfs:
    tick = ticker_df["Tick"].iloc[0]
    for col in ("Close", "Change"):
        df[f"{tick}_{col}"] = ticker_df.set_index("Date")[col]
df.tail()
s_and_p_df = df[["SXR8.DE_Close"]].copy(deep=True)

In [79]:
# Create column to identify ticker with minimum percent change in each row:
pct_change_cols = [c for c in df.columns if c.endswith("_Change")]
df["min_pct_change"] = df[pct_change_cols].idxmin(axis=1)

# Add the close value for the minimum identified ticker:
df["min_ticker_close_value"] = df.apply(
    lambda row: row[row["min_pct_change"].replace("Change", "Close")], axis=1)

df[["min_pct_change", "min_ticker_close_value"]].tail()

,min_pct_change,min_ticker_close_value
2023-05-19,VFEM.AS_Change,49.889999
2023-05-22,TIUP.DE_Change,100.330002
2023-05-23,ROBO_Change,54.220001
2023-05-24,AIAG.L_Change,1143.500000
2023-05-25,NBTK.DE_Change,39.040001


In [80]:
# Divide min_ticker_close_value by daily invest amount to calculate number of shares bought:
df["shares_bought"] = DAILY_INVEST / df["min_ticker_close_value"]
df[["min_pct_change", "min_ticker_close_value", "shares_bought"]].tail()

,min_pct_change,min_ticker_close_value,shares_bought
2023-05-19,VFEM.AS_Change,49.889999,200.440973
2023-05-22,TIUP.DE_Change,100.330002,99.671084
2023-05-23,ROBO_Change,54.220001,184.433784
2023-05-24,AIAG.L_Change,1143.500000,8.745081
2023-05-25,NBTK.DE_Change,39.040001,256.147535


In [81]:
# Calculate total shares bought in each ticker:
shares_bought = df.groupby(["min_pct_change"])["shares_bought"].sum()
shares_bought = {k.replace("_Change", ""): v for k, v in shares_bought.items()}
shares_bought

{'2B78.DE': 274311.9303552019,
 'AIAG.L': 1317.869944215018,
 'DXSA.DE': 472786.722589045,
 'IPRP.AS': 354467.67819743673,
 'NBTK.DE': 29528.54680089079,
 'ROBO': 139027.2788654988,
 'SXR8.DE': 41734.35329484815,
 'TIUP.DE': 29521.33425301259,
 'VFEM.AS': 99514.7952312188,
 'VWRL.AS': 9611.616851074152,
 'XMLH.DE': 63386.1338518698}

In [82]:
# Calculate final close prices of tickers:
final_close_prices = {
    k.replace("_Close", ""): v
    for k, v in df.ffill().iloc[-1].items() 
    if "_Close" in k
}
final_close_prices

{'AIAG.L': 1163.5,
 'NBTK.DE': 39.040000915527344,
 'XMLH.DE': 11.027999877929688,
 'ROBO': 54.170101165771484,
 'TIUP.DE': 100.92500305175781,
 'VWRL.AS': 99.04000091552734,
 'VFEM.AS': 49.529998779296875,
 'DXSA.DE': 20.18000030517578,
 'SXR8.DE': 401.5899963378906,
 'IPRP.AS': 23.584999084472656,
 '2B78.DE': 6.918000221252441}

In [83]:
# Calculate final value of holdings:
final_value = {k: v * final_close_prices[k] for k, v in shares_bought.items()}
final_value

{'2B78.DE': 1897689.994889471,
 'AIAG.L': 1533341.6800941734,
 'DXSA.DE': 9540836.206129985,
 'IPRP.AS': 8360119.865761694,
 'NBTK.DE': 1152794.4941409684,
 'ROBO': 7531121.760945994,
 'SXR8.DE': 16760098.786842303,
 'TIUP.DE': 2979440.749577258,
 'VFEM.AS': 4928967.686324245,
 'VWRL.AS': 951934.541730082,
 'XMLH.DE': 699022.276380855}

In [84]:
total_final_value = sum(final_value.values()) / 100
print(f"Final Value: {'€{:,.2f}'.format(total_final_value)}")
amount_invested = DAILY_INVEST * len(df) / 100
print(f"Amount Invested: {'€{:,.2f}'.format(amount_invested)}")
pct_change = (total_final_value - amount_invested) / amount_invested
print(f"{round(pct_change * 100, 3)}% change over {NO_YEARS} years")

Final Value: €563,353.68
Amount Invested: €396,500.00
42.082% change over 20 years


In [85]:
s_and_p_df["amt"] = DAILY_INVEST / s_and_p_df["SXR8.DE_Close"]
s_and_p_df = s_and_p_df[s_and_p_df["SXR8.DE_Close"].notna()]
s_and_p_df["value"] = s_and_p_df["amt"] * final_close_prices["SXR8.DE"]

total_final_value = sum(s_and_p_df["value"]) / 100
print(f"Final Value: {'€{:,.2f}'.format(total_final_value)}")

amount_invested = DAILY_INVEST * len(s_and_p_df) / 100
print(f"Amount Invested: {'€{:,.2f}'.format(amount_invested)}")
pct_change = (total_final_value - amount_invested) / amount_invested
print(f"{round(pct_change * 100, 3)}% change over {NO_YEARS} years")

Final Value: €834,033.56
Amount Invested: €330,600.00
152.279% change over 20 years
